In [7]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

     |████████████████████████████████| 4.3MB 5.0MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 5.1MB/s 


In [8]:
import pandas  as pd
import numpy  as  np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

import eli5
from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


# Load data

In [35]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

# Code from day 3

In [36]:
# remove EUR
df = df[ df['price_currency'] != 'EUR' ]

# format features
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[ feat ][0], list): continue

  factorized_values = df[ feat ].factorize()[0]
  # avoid columns named like 'feat___cat___cat'
  if SUFFIX_CAT in feat:
    df[ feat ] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]

# initial model

X  =  df[  cat_feats  ].values
y = df[ 'price_value' ].values

model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, X, y, cv = 5, scoring='neg_mean_absolute_error')
np.mean(scores), np.std(scores)

(-19587.598553433912, 290.2973424674993)

In [0]:
# function for model running
def run_model(model, feats):
  X  =  df[  feats  ].values
  y = df[ 'price_value' ].values

  scores = cross_val_score(model, X, y, cv = 5, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [16]:
run_model(model, cat_feats)

(-19691.1873923112, 146.80814532571364)

# Random Forest

In [20]:
rf_model = RandomForestRegressor(max_depth=5, n_estimators=50, random_state=0)
run_model(rf_model, cat_feats)

(-18963.010502655197, 248.0777235628634)

# XGBoost

In [21]:
xgb_params = {
    'max_depth' : 5,
    'n_estimators' : 50,
    'learning_rate': 0.1,
    'seed':0
}
xgb_model = xgb.XGBRegressor(**xgb_params)
run_model(xgb_model, cat_feats)

[06:54:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:54:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:54:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:55:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:55:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13092.898650168147, 248.10266039064473)

In [22]:
xgb_model.fit(X, y)

imp = PermutationImportance(xgb_model, random_state=0).fit(X, y)
eli5.show_weights(imp, feature_names=cat_feats)

[06:57:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1286 ± 0.0010,param_napęd__cat
0.1113 ± 0.0031,param_rok-produkcji__cat
0.1026 ± 0.0020,param_stan__cat
0.0597 ± 0.0021,param_faktura-vat__cat
0.0523 ± 0.0018,param_moc__cat
0.0440 ± 0.0015,param_skrzynia-biegów__cat
0.0277 ± 0.0005,feature_kamera-cofania__cat
0.0243 ± 0.0007,param_marka-pojazdu__cat
0.0241 ± 0.0004,param_typ__cat
0.0167 ± 0.0007,param_pojemność-skokowa__cat


In [0]:
feats = ['param_napęd__cat','param_rok-produkcji__cat','param_stan__cat','param_faktura-vat__cat','param_moc__cat',\
         'param_skrzynia-biegów__cat','feature_kamera-cofania__cat','param_marka-pojazdu__cat','param_typ__cat',\
         'param_pojemność-skokowa__cat','feature_wspomaganie-kierownicy__cat','seller_name__cat','param_wersja__cat',\
         'param_model-pojazdu__cat','feature_światła-led__cat','feature_asystent-pasa-ruchu__cat','param_kod-silnika__cat',\
         'feature_system-start-stop__cat','feature_regulowane-zawieszenie__cat','feature_łopatki-zmiany-biegów__cat']

In [24]:
# run model on limited number of features
run_model(xgb_model, feats)

[07:09:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:09:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:09:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:09:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:09:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13470.002051902518, 127.96491025445172)

# Features

In [38]:
# Convert strings to integers and none to -1
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))

# replace param_rok-produkcji_cat with modified param_rok-produkcji
feats = ['param_napęd__cat','param_rok-produkcji','param_stan__cat','param_faktura-vat__cat','param_moc__cat',\
         'param_skrzynia-biegów__cat','feature_kamera-cofania__cat','param_marka-pojazdu__cat','param_typ__cat',\
         'param_pojemność-skokowa__cat','feature_wspomaganie-kierownicy__cat','seller_name__cat','param_wersja__cat',\
         'param_model-pojazdu__cat','feature_światła-led__cat','feature_asystent-pasa-ruchu__cat','param_kod-silnika__cat',\
         'feature_system-start-stop__cat','feature_regulowane-zawieszenie__cat','feature_łopatki-zmiany-biegów__cat']

# run model on modified features
run_model(xgb_model, feats)

[07:24:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:24:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:24:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:24:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:24:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11298.433199124667, 158.4827113684511)

In [39]:
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]))
# replace param_moc_cat with modified param_moc
feats = ['param_napęd__cat','param_rok-produkcji','param_stan__cat','param_faktura-vat__cat','param_moc',\
         'param_skrzynia-biegów__cat','feature_kamera-cofania__cat','param_marka-pojazdu__cat','param_typ__cat',\
         'param_pojemność-skokowa__cat','feature_wspomaganie-kierownicy__cat','seller_name__cat','param_wersja__cat',\
         'param_model-pojazdu__cat','feature_światła-led__cat','feature_asystent-pasa-ruchu__cat','param_kod-silnika__cat',\
         'feature_system-start-stop__cat','feature_regulowane-zawieszenie__cat','feature_łopatki-zmiany-biegów__cat']
         
# run model on modified features
run_model(xgb_model, feats)

[07:24:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:24:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:25:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:25:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:25:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9659.191162182877, 62.65812422636104)

In [44]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split('cm3')[0].replace(' ', '')))
# replace param_pojemność-skokowa_cat with modified param_pojemność-skokowa
feats = ['param_napęd__cat','param_rok-produkcji','param_stan__cat','param_faktura-vat__cat','param_moc',\
         'param_skrzynia-biegów__cat','feature_kamera-cofania__cat','param_marka-pojazdu__cat','param_typ__cat',\
         'param_pojemność-skokowa','feature_wspomaganie-kierownicy__cat','seller_name__cat','param_wersja__cat',\
         'param_model-pojazdu__cat','feature_światła-led__cat','feature_asystent-pasa-ruchu__cat','param_kod-silnika__cat',\
         'feature_system-start-stop__cat','feature_regulowane-zawieszenie__cat','feature_łopatki-zmiany-biegów__cat']
         
# run model on modified features
run_model(xgb_model, feats)

[07:26:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:26:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:26:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:27:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:27:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9506.237090522724, 58.231213251218776)

In [45]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   day3_matrix2_simple_model.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	day4_matrix2_xgboost.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
